# Title: Security Metric Behavioral Analysis 
## Purpose: Characterize Metrics in Action
## Author: mjz

`./run_boromir.py --bigquery_table='cloud-performance-tool:test_notebook_bq.test2'`


## Background


In [1]:
%matplotlib inline

#------ python libs ----------
import logging
import os
SEP = os.path.sep
import sys
import uuid
import pathlib

#------ notebook libs ----------
import matplotlib.pyplot as plt
import networkx as nx
import graphviz
import IPython.display
import numpy as np
# import scipy.stats as st
# import scipy.special
# # bokeh plotting setup
# import bokeh.io
# import bokeh.plotting
# import bokeh.application
# import bokeh.application.handlers
# bokeh.io.output_notebook()
# notebook_url = 'localhost:8888'

import os
from google.cloud import bigquery
%reload_ext google.cloud.bigquery

import pandas as pd
import pandas_gbq


#------ project libs ----------
py_mulval_path = r'/opt/projects/diss/py-mulval/src'
sys.path.append(py_mulval_path)

# Just importing for their flags right now
import py_mulval.py_mulval as py_mulval
import py_mulval.boromir as boromir
import py_mulval.log_util as log_util
import py_mulval.mulpy as mulpy
import py_mulval.metrics as secmet
import py_mulval.attack_graph as attack_graph
from py_mulval.mulval_fact_graph import FactGraph
from py_mulval.attack_graph import AttackGraph

from py_mulval import flags
FLAGS = flags.FLAGS
FLAGS([sys.argv[0]]) # dont expect cli args from jupyter

['/opt/projects/diss/jupyter_nbs/py36/lib/python3.6/site-packages/ipykernel_launcher.py']

In [2]:
ds = 'test_notebook_bq'
tab = 'test1'
proj = 'cloud-performance-tool'
tbl2_full = '`cloud-performance-tool.test_notebook_bq.test2`'
key_loc = '/home/toor/.google/smu-benchmarking-7de5dd11902f.json'

# Setup


In [3]:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_loc # bq_client needs this (svc_acct auth)
# bigquery_client = bigquery.Client(project=proj) # 

pandas_gbq.context.project = proj
pandas_gbq.context.dialect = 'standard'

In [ ]:
! 

In [4]:
%%bigquery
SELECT * FROM `cloud-performance-tool.test_notebook_bq.test2`
where labels != ''
limit 3

/opt/projects/diss/jupyter_nbs/py36/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/projects/diss/jupyter_nbs/py36/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see http

Executing query with job ID: 6eee9df2-3326-493d-9ed2-8629743b292d
Query executing: 0.80s


ERROR:
 404 Not found: Table cloud-performance-tool:test_notebook_bq.test2 was not found in location US

(job ID: 6eee9df2-3326-493d-9ed2-8629743b292d)

                 -----Query Job SQL Follows-----                  

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT * FROM `cloud-performance-tool.test_notebook_bq.test2`
   2:where labels != ''
   3:limit 3
    |    .    |    .    |    .    |    .    |    .    |    .    |


In [8]:
%%bigquery df
SELECT * FROM `cloud-performance-tool.test_notebook_bq.test2`



In [44]:
%%bash
pip freeze

absl-py==0.9.0
antlr4-python3-runtime==4.7.2
astor==0.8.1
attrs==19.3.0
backcall==0.1.0
bleach==3.1.0
blinker==1.4
bokeh==1.4.0
cachetools==4.0.0
certifi==2019.11.28
chardet==3.0.4
ciso8601==2.1.3
contextlib2==0.6.0.post1
cowsay==2.0.3
cycler==0.10.0
decorator==4.4.1
defusedxml==0.6.0
entrypoints==0.3
gast==0.2.2
get==2019.4.13
google-api-core==1.16.0
google-auth==1.11.2
google-auth-oauthlib==0.4.1
google-cloud-bigquery==1.24.0
google-cloud-core==1.3.0
google-pasta==0.1.8
google-resumable-media==0.5.0
googleapis-common-protos==1.51.0
graphviz==0.13.2
grpcio==1.27.2
h5py==2.10.0
idna==2.8
importlib-metadata==1.1.0
influxdb==5.2.3
ipykernel==5.1.3
ipython==7.10.1
ipython-genutils==0.2.0
jedi==0.15.1
Jinja2==2.10.3
joblib==0.14.1
json5==0.8.5
jsonschema==3.2.0
jupyter-client==5.3.4
jupyter-contrib-core==0.3.3
jupyter-contrib-nbextensions==0.5.1
jupyter-core==4.6.1
jupyter-highlight-selected-word==0.2.0
jupyter-latex-envs==1.4.6
jupyter-nbextensions-configurator==0.4.1
jupyterlab==1.2.3
ju

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 11 columns):
sample_uri      38 non-null object
run_uri         38 non-null object
owner           38 non-null object
official        38 non-null bool
timestamp       38 non-null float64
unit            38 non-null object
test            38 non-null object
product_name    38 non-null object
value           38 non-null float64
labels          38 non-null object
metric          38 non-null object
dtypes: bool(1), float64(2), object(8)
memory usage: 3.1+ KB


In [10]:
df.head()

,sample_uri,run_uri,owner,official,timestamp,unit,test,product_name,value,labels,metric
0,31302b02-7e1e-445a-90d0-32393bb84495,c3b23282,cat-dog,False,1.582659e+09,effs,metf_tm,py_mulval,0.616000,"|citation:Rodolphe Ortalo, Yves Deswarte, and ...",metf_tm
1,07ba50de-e979-4eaa-8dd9-6c9795ab2c53,c3b23282,cat-dog,False,1.582659e+09,weeks,mttf,py_mulval,52.033500,"|attack_graph_name:single_host_1|,|citation:Ma...",mttf
2,c0d62171-7cdf-4c5a-a67e-a5495977e078,c3b23282,cat-dog,False,1.582659e+09,weeks,shortest_path_cumulative,py_mulval,153.467000,"|all_shortest_paths:[('0', '13', '5', '3', '1'...",shortest_path_cumulative
3,960a2bb7-5443-47da-80ad-dc2de8164323,c3b23282,cat-dog,False,1.582659e+09,seconds,metf_tm,py_mulval,0.021333,,End to End Runtime
4,3b0887b3-03f3-4126-9d46-3b9480694250,c3b23282,cat-dog,False,1.582659e+09,seconds,mttf,py_mulval,0.011740,,End to End Runtime


In [11]:
%%bigquery df
#standardSQL
SELECT thedate, test, value, unit, labels
#   concat(sending_region, '-', receiving_region) as regions
FROM (
  SELECT
#     value,
    TIMESTAMP_TRUNC(TIMESTAMP_MICROS(CAST(timestamp * 1000000 AS int64)), DAY) AS thedate,
    test, value, unit, labels
#     REGEXP_EXTRACT(labels, r'\|vm_1_zone:(.*?)\|') AS sending_zone,
#     REGEXP_EXTRACT(labels, r'\|vm_2_zone:(.*?)\|') AS receiving_zone,
#     REGEXP_EXTRACT(labels, r'\|vm_1_zone:us-(.*?)-.*?\|') AS sending_region,
#     REGEXP_EXTRACT(labels, r'\|vm_2_zone:us-(.*?)-.*?\|') AS receiving_region
  FROM
    `cloud-performance-tool.test_notebook_bq.test2`)
#   WHERE
#     test = 'ping'
#     AND metric = 'Average Latency' )setup
# WHERE
  #sending_zone != receiving_zone
#   sending_region != receiving_region
# group by thedate, sending_region, receiving_region


In [15]:
print(df.labels[0])
df.head()

|citation:None|,|cite_key:None|,|metric_name:None|,|metric_summary:None|,|metric_unit:None|,|metric_usage:None|,|run_number:0|


,thedate,test,value,unit,labels
0,2020-02-25 00:00:00+00:00,metf_tm,0.6160,effs,"|citation:None|,|cite_key:None|,|metric_name:N..."
1,2020-02-25 00:00:00+00:00,metf_ml,0.6160,effs,"|citation:None|,|cite_key:None|,|metric_name:N..."
2,2020-02-25 00:00:00+00:00,num_paths,2.0000,paths,"|all_paths_original:[[""18"", ""16"", ""15"", ""14"", ..."
3,2020-02-25 00:00:00+00:00,mttf,52.0335,weeks,"|attack_graph_name:single_host_1|,|citation:Ma..."
4,2020-02-25 00:00:00+00:00,shortest_path_cumulative,153.4670,weeks,"|all_shortest_paths:[('0', '13', '5', '3', '1'..."


# Range Tests

Identify what min and max values a measurement will assume under given conditions

## MTTF



### Fixed CVSS

Run MTTF against the same AG, fixing transition times from lowest (easiest) to highest (longest)

In [19]:
%%bash
/opt/projects/diss/py-mulval/src/run_boromir.py --benchmarks=mttf --benchmark_config_file=mttf.yml --bigquery_table='cloud-performance-tool:test_notebook_bq.test2'


-------------------------BOROMIR Results Summary-------------------------
MTTF:
  attack_graph_name="single_host_1" citation="Marc Dacier, Yves Deswarte, and Mohamed Kaâniche. 1996. 
Quantitative assessment of operational security: Models and tools. 
Information Systems Security, ed. by SK Katsikas and D. Gritzalis, London, 
Chapman & Hall (1996), 179–86." cite_key="dacier1996" metric_name="mttf" metric_summary=""Mean Time To Failure: Determines the survival function complement from reliability engineering. 
" metric_unit="weeks" run_number="0" usage="Accepts an attack graph and the node to start from, or looks for 
the origin if no node provided"
  mttf                                  0.000800 weeks                         
  End to End Runtime                    0.013899 seconds                       
  mttf                                  0.080000 weeks                         
  End to End Runtime                    0.010677 seconds                       
  mttf                 

2020-02-26 15:14:05,771 cfd2714b MainThread INFO     Verbose logging to: /tmp/mulpy/runs/cfd2714b/pkb.log
2020-02-26 15:14:05,771 cfd2714b MainThread INFO     PerfKitBenchmarker version: unknown
2020-02-26 15:14:05,772 cfd2714b MainThread INFO     Flag values:
--benchmark_config_file=mttf.yml
--bigquery_table=cloud-performance-tool:test_notebook_bq.test2
--benchmarks=mttf
2020-02-26 15:14:05,788 cfd2714b MainThread INFO     Running: ssh-keygen -t rsa -N  -m PEM -q -f /tmp/mulpy/runs/cfd2714b/py_mulval_keyfile
2020-02-26 15:14:05,832 cfd2714b MainThread INFO     Setting --max_concurrent_threads=200.
2020-02-26 15:14:05,852 cfd2714b MainThread WARNING  The key "secmet_plot_intermediate_graphs" was not in the default config, but was in user overrides. This may indicate a typo.
2020-02-26 15:14:05,852 cfd2714b MainThread WARNING  The key "secmet_map_scores" was not in the default config, but was in user overrides. This may indicate a typo.
2020-02-26 15:14:05,852 cfd2714b MainThread WARNIN

In [33]:
%%bigquery df
#standardSQL
SELECT thedate, test, value, unit, metric, labels
#   concat(sending_region, '-', receiving_region) as regions
FROM (
  SELECT
#     value,
    *, TIMESTAMP_MICROS(CAST(timestamp * 1000000 AS int64)) AS thedate,
#     test, value, unit, labels
#     REGEXP_EXTRACT(labels, r'\|vm_1_zone:(.*?)\|') AS sending_zone,
#     REGEXP_EXTRACT(labels, r'\|vm_2_zone:(.*?)\|') AS receiving_zone,
#     REGEXP_EXTRACT(labels, r'\|vm_1_zone:us-(.*?)-.*?\|') AS sending_region,
#     REGEXP_EXTRACT(labels, r'\|vm_2_zone:us-(.*?)-.*?\|') AS receiving_region
  FROM
    `cloud-performance-tool.test_notebook_bq.test2`)
  WHERE
    test = 'mttf'
    AND metric = 'mttf'
    and run_uri = 'cfd2714b'
# WHERE
  #sending_zone != receiving_zone
#   sending_region != receiving_region
# group by thedate, sending_region, receiving_region


In [39]:
import matplotlib.pyplot as plt

%matplotlib inline

In [43]:
df.head()

,thedate,test,value,unit,metric,labels
0,2020-02-26 21:14:06.043697+00:00,mttf,0.0008,weeks,mttf,"|attack_graph_name:single_host_1|,|citation:Ma..."
1,2020-02-26 21:14:06.055289+00:00,mttf,0.0800,weeks,mttf,"|attack_graph_name:single_host_1|,|citation:Ma..."
2,2020-02-26 21:14:06.066213+00:00,mttf,0.0800,weeks,mttf,"|attack_graph_name:single_host_1|,|citation:Ma..."
3,2020-02-26 21:14:06.077121+00:00,mttf,0.8000,weeks,mttf,"|attack_graph_name:single_host_1|,|citation:Ma..."
4,2020-02-26 21:14:06.088149+00:00,mttf,0.8000,weeks,mttf,"|attack_graph_name:single_host_1|,|citation:Ma..."


In [ ]:
df 